In [ ]:
import pandas as pd

def getStockDetails(name,isAnnual):
    sections = ['stock-financial-report','stock-financial-ratio']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    # headers = []
    quarters = []
    data = []
    toggled = False
    for section in sections:
            obj = {}
            keys = []
            # names = []
            headerElements = driver.find_element(By.CSS_SELECTOR,
        f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            # if len(headers) == 0:
            #     for div in BeautifulSoup(headerElements.get_attribute('innerHTML'), 'html.parser').find_all('div', {'class': 'topic'}):
            #         headers.append(div.get_text())
            if not toggled and isAnnual:
                toggled = True
                e = driver.find_element(By.CLASS_NAME,'toggle')
                webdriver.ActionChains(driver).move_to_element(e).click(e).perform()
            for topic in headerElements.find_elements(By.CLASS_NAME,'topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                keys.append(key)
                # name = BeautifulSoup(topic.get_attribute('innerHTML'), 'html.parser').get_text()
                # names.append(name)
                
            contentElements = driver.find_element(By.CSS_SELECTOR, 
                                f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]}"
                obj[key] = values
                data.append(obj)
    # return (headers,quarters,data)
    return (quarters,data)

def getFinancialDetails(symbol,isAnnual):
    # headers,quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=isAnnual)
    quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=isAnnual)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data['Period'] = quarterlyOrAnnual
    data['Symbol'] = symbol
    df = pd.DataFrame(data) 
    df = df.iloc[:-1]
    df = df.drop(df.columns[[5, 6, 8, 9, 10, 11, 15, 16, 17, 18, 19]], axis=1)
    return df

In [2]:
import requests
response = requests.get('https://www.finnomena.com/stock/AACI')

if response.status_code == 404:
    pass
else:
    print("Page exists")